In [1]:
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

import numpy as np

from noise import pnoise2
from random import randint

import time

In [2]:
def get_terrain(shape, scale, noise_kwargs):
    terrain = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            terrain[i][j] = pnoise2(i / scale, j / scale, **noise_kwargs)
    
    return terrain

In [40]:
# Generate 2D Perlin noise at coordinates (x, y)
shape = height, width = 256, 256
scale = 500.0

# pnoise_kwargs = {"octaves":3,           # Number of noise layers
#                  "persistence": 0.6,    # Amplitude of each successive octave
#                  "lacunarity": 2.0,     # Frequency multiplier between octaves
#                  "base": 979}           # Base seed for the noise (optional)
pnoise_kwargs = {"octaves":5,           # Number of noise layers
                 "persistence": 0.6,    # Amplitude of each successive octave
                 "lacunarity": 1.8,     # Frequency multiplier between octaves
                 "base": 74,            # Base seed for the noise (optional)
                 } 

time_start = time.process_time()
terrain = get_terrain(shape, scale, pnoise_kwargs)
print(f"Time to create terrain matrix: {(time.process_time()-time_start):.4f}")


time_start = time.process_time()
# Gradiente
grad_terrain = np.gradient(terrain)
print(f"Time to get the gradient matrix: {(time.process_time()-time_start):.4f}")


time_start = time.process_time()
# Normalize terrain values to 0–1
normalized_terrain = (terrain - terrain.min()) / (terrain.max() - terrain.min())
print(f"Time to get normalize matrix: {(time.process_time()-time_start):.4f}")


time_start = time.process_time()
# Colored elevation map
plt.imshow(normalized_terrain, cmap='terrain')
plt.title("Colored Terrain Elevation")
plt.colorbar()

ax = plt.figure().add_subplot(projection='3d')

x = y = np.linspace(0,height, height)
X, Y = np.meshgrid(x, y)  # Create meshgrid

ax.plot_surface(X,Y, normalized_terrain, cmap='terrain', linewidth=0, antialiased=False)

print(f"Time to plot: {(time.process_time()-time_start):.4f}")

plt.show()

Time to create terrain matrix: 0.0673
Time to get the gradient matrix: 0.0006
Time to get normalize matrix: 0.0002
Time to plot: 0.0986


In [41]:
dt = 0.5
g = 10
friction = .01
erosion_rate = 1
initial_sediment = 0

class droplet():
    def __init__(self, height, width):
        # Dsp esto lo puede tener una clase que sea el terreno
        self.height = height
        self.width = width

        # Variables dinamicas de la gota
        self.inbounds = True

        self.sediment = initial_sediment # masa de sedimento

        self.ipos = np.random.randint([0,0],[height-1, width-1], 2)
        self.pos = np.array(self.ipos, float)
        self.vel = np.zeros(2)

        # self.path = [self.pos]
        self.max_speed = 0

    def is_inbounds(self):
        if self.pos[0] >= self.height - 1 or self.pos[1] >= self.width - 1:
            return False
        elif self.pos[0] < 0 or self.pos[1] < 0:
            return False
        else: 
            return True

    def step(self, terrain, grad_terrain, dt):
        
        if not self.inbounds:
            print("step not executed, out of bounds")
            return
        
        self.ipos = np.array(self.pos, int)

        # Dinamica
        slope = np.array((grad_terrain[0][*self.ipos],grad_terrain[1][*self.ipos]))

        self.vel -= dt*slope*g
        self.pos += dt*self.vel
        self.vel *= (1-dt*friction)
        
        # Check q esta en la grilla
        if not self.is_inbounds():
            # print("out of bounds")
            self.inbounds = False

            # si sale del mapa deposita el sedimento en la ultima posición
            # para que no haya perdidas
            terrain[*self.ipos] += self.sediment

            return 
        # else:
        #     self.path.append(np.array(self.pos, copy = True))

        # Erosion
        if False: # (self.ipos==np.astype(self.pos, int)).all(): # No hay erosión si no cambia de celda (cuestionable)
            return
        else:
            speed = np.sqrt(self.vel.dot(self.vel))
            if speed > self.max_speed:
                self.max_speed = speed

            erosion = speed * (terrain[*self.ipos]-terrain[*np.array(self.pos,int)])
            if erosion < 0 : erosion = 0 # Solo hay erosion cuando baja (cuestionable)

            s_diff = dt*erosion_rate * (erosion - self.sediment)

            if terrain[*np.array(self.pos,int)] - (terrain[*self.ipos]-s_diff) < 0:
                self.sediment += s_diff
                terrain[*self.ipos] -= s_diff
            else:
                self.sediment += terrain[*self.ipos]-terrain[*np.array(self.pos,int)]
                terrain[*self.ipos] = terrain[*np.array(self.pos,int)]

In [66]:
N_droplets = 5000
batch_size = 50

max_droplet_steps = 1000

terrain_copy = terrain.copy()

paths = []

step_times = []
batch_times = []
steps = []
speeds = []

for i in range(N_droplets//batch_size):
        time_start = time.process_time()
        
        # Hago un Batch antes de actualizar el mapa de gradiente
        for j in range(batch_size):
                pepe = droplet(height, width)
                i=0
                path = [pepe.pos.copy()]
                while pepe.inbounds and i < max_droplet_steps:

                        temp_time_start = time.process_time()  

                        pepe.step(terrain = terrain_copy, grad_terrain=grad_terrain, dt = dt)

                        step_times.append(time.process_time()-temp_time_start)
                        path.append(pepe.pos.copy())
                        # print(pepe.pos, pepe.ipos, pepe.path[-1])
                        i+=1 
                steps.append(i)
                speeds.append(pepe.max_speed)
                paths.append(path)
        grad_terrain = np.gradient(terrain_copy)
        batch_times.append(time.process_time()-time_start)
        
print(f"mean step time: {np.mean(step_times)}")
print(f"Batch time: {np.mean(batch_times)}")

print(f"mean steps: {np.mean(steps)}")
print(f"mean max speed: {np.mean(speeds)}")
print(f"absolute max speed: {np.max(speeds)}")

normalized_terrain = (terrain_copy - terrain_copy.min()) / (terrain_copy.max() - terrain_copy.min())

mean step time: 1.6155219974939326e-05
Batch time: 0.4461014352000005
mean steps: 522.2404
mean max speed: 0.6510915729046928
absolute max speed: 1.3231117750734103


In [73]:
fig, ax = plt.subplots()

map = ax.imshow(terrain, cmap='terrain')
# plt.colorbar(map, ax = ax)
ax.set_title("Colored Terrain Elevation")
ax.set(xticks=[], yticks=[])

for path in paths[0:300]:
    path_x, path_y = [a[1] for a in path],[a[0] for a in path]
    ax.plot(path_x, path_y, ".")

plt.show()

In [74]:
fig, [ax,axx] = plt.subplots(1,2, dpi = 150)

map = ax.imshow(terrain_copy, cmap='terrain')
# plt.colorbar(map, ax = ax)
ax.set(title = "Colored Terrain Elevation", xticks=[], yticks=[])

map = axx.imshow(terrain_copy-terrain, cmap='terrain')
# plt.colorbar(map, ax = axx)
axx.set(title = "Colored Terrain Difference", xticks=[], yticks=[])

ax = plt.figure().add_subplot(projection='3d')

x = y = np.linspace(1,height,height)
X, Y = np.meshgrid(x, y)  # Create meshgrid

Delta_terrain = terrain_copy - terrain

ax.plot_surface(X,Y, terrain + 3*Delta_terrain, cmap='terrain', linewidth=0, antialiased=False)
# ax.set_box_aspect((4,4,1))

plt.show()

In [19]:
fig, ax = plt.subplots(1,1, dpi = 150)

ls = matplotlib.colors.LightSource(azdeg=315, altdeg=45)
map = ax.imshow(ls.hillshade(terrain_copy, vert_exag=5), cmap='gray')#, cmap='terrain')
# plt.colorbar(map, ax = ax)
ax.set(title = "Colored Terrain Elevation", xticks=[], yticks=[])

fig.show()

In [73]:
ax = plt.figure().add_subplot(projection='3d')

x = y = np.linspace(1,height,height)
X, Y = np.meshgrid(x, y)  # Create meshgrid

ax.plot_surface(X,Y, terrain_copy-terrain, cmap='terrain', linewidth=0, antialiased=False)
# ax.set_box_aspect((4,4,1))

plt.show()

In [ ]:
ax = plt.figure().add_subplot(projection='3d')

x = y = np.linspace(1,height,height)
X, Y = np.meshgrid(x, y)  # Create meshgrid

ax.plot_surface(X,Y, terrain_copy, linewidth=0, antialiased=False, shade = True)
# ax.set_box_aspect((4,4,1))

plt.show()

## Particle trayectories

In [ ]:
fig, ax = plt.subplots()

map = ax.imshow(terrain, cmap='terrain')
# plt.colorbar(map, ax = ax)
ax.set_title("Colored Terrain Elevation")
ax.set(xticks=[], yticks=[])

for path in paths:
    path_x, path_y = [a[1] for a in path],[a[0] for a in path]
    ax.plot(path_x, path_y, ".")

plt.show()

in loop
path obtained
in loop
path obtained
in loop
path obtained
in loop
path obtained
in loop
path obtained
